In [ ]:
!pip install fosforml
!pip install category-encoders==2.6.1

In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from fosforml import *
from fosforml.constants import MLModelFlavours

In [ ]:
from refractio import snowflake
cars = snowflake.get_dataframe("CAR_NEW100MB")
cars

In [2]:
cars = pd.read_csv("car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [3]:
cars.columns = [x.upper() for x in cars.columns]
cars

,BUYING_NUM,MAINTENANCE_NUM,DOORS_NUM,PERSONS_NUM,LUGBOOT_NUM,SAFETY_NUM,CLASS_NUM
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [4]:
Y = cars['CLASS_NUM']
X = cars.drop (['CLASS_NUM'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [ ]:
X_train.head(3000).to_csv("/notebooks/notebooks/cars_X_train.csv", index = False)

In [5]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [ ]:
import sys
m_size = sys.getsizeof(model)
m_size

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(model, "/notebooks/notebooks/CAR_NEW200MB")

In [ ]:
X_train.head(1)

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [ ]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

In [ ]:
register_model(model, 
               score, 
               name='car_evaluation_new', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=False, 
               kyd_score = False
              )

In [ ]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()


In [ ]:
register_model(
  model_obj=model,
  session=session,
  x_train=X_train,
  y_train=y_train,
  x_test=X_test,
  y_test=y_test,
  y_pred=y_pred,
  source="Notebook",
  dataset_name="car_new",
  dataset_source="InMemory",
  name="PRAKHAR_CAR_EVALUATION",
  description="This is a sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

In [ ]:
y_pred

In [ ]:
!pip freeze | grep -i sci

In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry as model_registry
import time